In [1]:
import pandas as pd
data=pd.read_csv('/content/drive/MyDrive/Churn_Modelling.csv')

In [2]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [4]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
data.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

**Convert Geography,Gender column into numeric**

In [6]:
data['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [7]:
geo=pd.get_dummies(data['Geography'],drop_first=True)

In [8]:
gender=pd.get_dummies(data['Gender'],drop_first=True)

In [9]:
df=pd.concat([data,geo,gender],axis=1)

In [10]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Germany,Spain,Male
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,1,0


In [11]:
df.drop(['Geography','Gender'],axis=1,inplace=True)

In [12]:
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


In [17]:
x=df.drop('Exited',axis=1)
y=df["Exited"]
x.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


In [18]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=67)

In [24]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)


In [26]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout

In [27]:
classifier=Sequential()
classifier.add(Dense(units=7,kernel_initializer='he_uniform',activation='relu',input_dim=11))
classifier.add(Dense(units=7,activation='relu',kernel_initializer='he_uniform'))
classifier.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))

In [28]:
classifier.compile(optimizer='Adamax',loss='binary_crossentropy',metrics=['accuracy'])

In [29]:
model_history=classifier.fit(x_train,y_train,validation_split=0.33,batch_size=10,epochs=100)

Epoch 1/100
536/536 [==============================] - 2s 2ms/step - loss: 0.5183 - accuracy: 0.7839 - val_loss: 0.4840 - val_accuracy: 0.8039
Epoch 2/100
536/536 [==============================] - 1s 2ms/step - loss: 0.4878 - accuracy: 0.7929 - val_loss: 0.4691 - val_accuracy: 0.8039
Epoch 3/100
536/536 [==============================] - 1s 2ms/step - loss: 0.4754 - accuracy: 0.7929 - val_loss: 0.4609 - val_accuracy: 0.8039
Epoch 4/100
536/536 [==============================] - 1s 2ms/step - loss: 0.4670 - accuracy: 0.7923 - val_loss: 0.4555 - val_accuracy: 0.8035
Epoch 5/100
536/536 [==============================] - 1s 2ms/step - loss: 0.4609 - accuracy: 0.7931 - val_loss: 0.4519 - val_accuracy: 0.8031
Epoch 6/100
536/536 [==============================] - 1s 2ms/step - loss: 0.4560 - accuracy: 0.7932 - val_loss: 0.4489 - val_accuracy: 0.8042
Epoch 7/100
536/536 [==============================] - 1s 2ms/step - loss: 0.4518 - accuracy: 0.7936 - val_loss: 0.4457 - val_accuracy: 0.8050

In [30]:
y_pred=classifier.predict(x_test)

In [32]:
y_pred=(y_pred>0.5)

In [34]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

In [35]:
cm

array([[1524,   67],
       [ 204,  205]])

In [36]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_pred,y_test)

In [37]:
score

0.8645